In [1]:
import requests
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

# Pair Programming ETL Transformación II

En la lección de hoy aprendimos como crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

- metodo llamar API peninsular
- metodo llamar API ccaa
- metodo datetime
- metodo round
- juntar dataframes

In [2]:
class Extraccion: 
    def __init__(self, año_inicio, año_final):

        self.año_inicio = año_inicio
        self.año_final = año_final

    def llamada_API_peninsula(self):
        df_peninsula = pd.DataFrame()
        
        for i in range(self.año_inicio, (self.año_final + 1)):
                
                url1 = f'https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={i}-01-01T00:00&end_date={i}-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=peninsular&geo_ids=8741'
                response1 = requests.get(url=url1)
                if response1.status_code != 200:
                    print(response1.reason, response1.status_code)
                    break
                
                for x in range(len(response1.json()['included'])):
                    df = pd.DataFrame(response1.json()['included'][x]['attributes']['values'])
                    df['tipo_energia'] = response1.json()['included'][x]['attributes']['title']                
                    
                    df_peninsula = pd.concat([df_peninsula, df], axis=0)
                    df_peninsula['region'] = 'Península'
                    
        return df_peninsula
        
    def llamada_API_ccaa(self, **dicc_ccaa):
    
        df_ccaa = pd.DataFrame()
        
        for i in range(self.año_inicio, (self.año_final + 1)):
            
            for k, v in dicc_ccaa.items():

                url2 = f'https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={i}-01-01T00:00&end_date={i}-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={v}'
                response2 = requests.get(url=url2)
                if response2.status_code != 200:
                    print(response2.reason, response2.status_code)
                    break
                
                for x in range(len(response2.json()['included'])):
                    df = pd.DataFrame(response2.json()['included'][x]['attributes']['values'])
                    df['tipo_energia'] = response2.json()['included'][x]['attributes']['title']
                    df['region'] = k
                    df['cod_comunidad'] = v
                    
                    df_ccaa = pd.concat([df_ccaa, df], axis=0)
                    
        return df_ccaa
            
    def limpiar(self, dataframe):
        
        df_num = dataframe.select_dtypes(include=np.number)
        
        for col in df_num.columns:
            dataframe[col] = round(dataframe[col], 2)
            
        dataframe['datetime'] = dataframe['datetime'].apply(pd.to_datetime)
        
        dataframe['datetime'] = dataframe['datetime'].apply(lambda x: x.strftime('%Y-%m-%d'))
        
        return dataframe
    
    def juntar(self, dataframe1, dataframe2):
        df_final = pd.concat([dataframe1, dataframe2], axis=0, join='outer')
        return df_final

    def guardar(self, dataframe, formato, nombre_archivo, ruta):
        if formato == 'csv':
            dataframe.to_csv(f'{ruta}{nombre_archivo}.csv')
        elif formato == 'pickle':
            dataframe.to_pickle(f'{ruta}{nombre_archivo}.pkl')

In [3]:
energia_españa = Extraccion(2011, 2022)

In [4]:
df_peninsula = energia_españa.llamada_API_peninsula()

In [5]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [6]:
df_ccaa = energia_españa.llamada_API_ccaa(**cod_comunidades)

In [7]:
df_ccaa

,value,percentage,datetime,tipo_energia,region,cod_comunidad
0,2.029729e+05,1.000000,2011-01-01T00:00:00.000+01:00,No renovable,Ceuta,8744
0,3.589082e+03,0.016703,2011-01-01T00:00:00.000+01:00,Renovable,Melilla,8745
0,2.112900e+05,0.983297,2011-01-01T00:00:00.000+01:00,No renovable,Melilla,8745
0,1.099692e+07,0.284857,2011-01-01T00:00:00.000+01:00,Renovable,Andalucía,4
0,2.760811e+07,0.715143,2011-01-01T00:00:00.000+01:00,No renovable,Andalucía,4
...,...,...,...,...,...,...
0,6.813412e+06,0.798664,2022-01-01T00:00:00.000+01:00,No renovable,Canarias,8742
0,2.670446e+06,0.229165,2022-01-01T00:00:00.000+01:00,Renovable,Región de Murcia,21
0,8.982495e+06,0.770835,2022-01-01T00:00:00.000+01:00,No renovable,Región de Murcia,21
0,1.106679e+06,0.476786,2022-01-01T00:00:00.000+01:00,Renovable,La Rioja,20


In [8]:
df_final = energia_españa.juntar(df_peninsula, df_ccaa)

In [9]:
df_final.head(2)

,value,percentage,datetime,tipo_energia,region,cod_comunidad
0,8.583292e+07,0.324714,2011-01-01T00:00:00.000+01:00,Renovable,Península,NaN
0,1.785007e+08,0.675286,2011-01-01T00:00:00.000+01:00,No renovable,Península,NaN


In [10]:
df_final = energia_españa.limpiar(df_final)

In [11]:
energia_españa.guardar(df_final, 'pickle', 'energia_españa_2011-2022', 'datos/')

In [16]:
df_peninsula = energia_españa.limpiar(df_peninsula)

In [17]:
df_ccaa = energia_españa.limpiar(df_ccaa)

In [18]:
df_peninsula.to_csv('datos/energia_año_peninsula.csv')

In [19]:
df_ccaa.to_csv('datos/energia_año_ccaa.csv')